In [4]:
!pip install -U datasets pandas "numpy<2" matplotlib seaborn sentence-transformers

Defaulting to user installation because normal site-packages is not writeable


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 367.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 225.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, concatenate_datasets, Dataset
base_dataset = load_dataset("facebook/natural_reasoning")

In [ ]:
def change_box_by_reference_answer(batch):
    # 'question', 'reference_answer', 'responses'
    import re
    # \\\boxed{}
    final_answer_regex = re.compile(r"\\boxed{([^}]*)}")
    ignore_data = []
    for question, reference_answer, responses in zip(batch['question'], batch['reference_answer'], batch['responses']):
        # find the last \\boxed{} in the responses
        try:
            rex = re.compile(r"\\boxed{([^}]*)}")
            # find the last \\boxed{} in the responses
            last_boxed = None
            if len(responses) == 0 or responses is None:
                ignore_data.append(True)
                continue
            for response in responses:
                match = rex.search(response)
                if match:
                    last_boxed = match.group(1)
            if last_boxed is not None:
                ignore_data.append(False)
                if last_boxed != reference_answer and reference_answer != "" and reference_answer is not None:
                    # replace the last \\boxed{} with the reference_answer
                    for i, response in enumerate(responses):
                        responses[i] = rex.sub(f"\\boxed{{{reference_answer}}}", response)
            else:
                ignore_data.append(True)
        except Exception as e:
            print(e, question, reference_answer, responses)
            ignore_data.append(True)
    batch['ignore_data'] = ignore_data
    return batch

facebook_dataset = facebook_dataset.map(change_box_by_reference_answer, batched=True)

In [2]:
open_math = load_dataset("ai2-adapt-dev/openmath-2-math", split="train")

Generating train split: 100%|██████████| 11402286/11402286 [00:18<00:00, 602720.67 examples/s]


In [ ]:
#

Dataset({
    features: ['messages'],
    num_rows: 11402286
})

In [3]:
open_r1 = load_dataset("open-r1/OpenR1-Math-220k", "all", split="train")

Generating train split: 100%|██████████| 225129/225129 [00:15<00:00, 14876.94 examples/s]


In [9]:
# rename problem and solution to question and response
open_r1 = open_r1.rename_column("problem", "question")
open_r1 = open_r1.rename_column("solution", "response")

In [8]:
# split messages column into question and answer
def split_messages(example):
    example["question"] = example["messages"][0]["content"]
    example["answer"] = example["messages"][1]["content"]
    return example

open_math = open_math.map(split_messages, remove_columns=["messages"])

Map: 100%|██████████| 11402286/11402286 [03:40<00:00, 51615.27 examples/s]


In [10]:
base_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'response', '__index_level_0__'],
        num_rows: 717281
    })
})

In [49]:
base_dataset.filter(lambda x: x['question'] and x['answer'])

Filter:   0%|          | 0/2559963 [00:00<?, ? examples/s]

KeyError: 'answer'

In [41]:
# concat 3
concat_dataset = concatenate_datasets([base_dataset, open_r1])
# select only question and answer columns
concat_dataset = concat_dataset.select_columns(["question", "answer"])

# remove empty question and answers
concat_dataset = concat_dataset.filter(lambda x: x['question'] and x['answer'])

Filter:   0%|          | 0/2785092 [00:00<?, ? examples/s]

In [47]:
concat_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 225129
})

In [43]:
# remove duplicates
import pandas as pd
df = pd.DataFrame(concat_dataset)
# df = df.drop_duplicates(subset=["question"])

# cleaned_dataset = Dataset.from_pandas(df)

In [45]:
df

,question,answer
0,9.043. $\sqrt{x^{2}-x-12}<x$.,x\in[4;\infty)
1,14. Two circles $C_{1}$ and $C_{2}$ with cente...,48
2,1. Given $\triangle A B C$ with the three angl...,1
3,Given vectors $\overrightarrow{a}$ and $\overr...,"[\frac{\sqrt{3}}{2}, \frac{\sqrt{7}}{2}]"
4,"Let $A=\{x \mid x^2 - 3x + 2 < 0\}$, $B=\{x \m...","m \in [4, +\infty)"
...,...,...
225124,"## Task 2\n\nIn a basket, there are 5 blue and...",3
225125,Given the following equations: 2+4=6; 8+10+12=...,\text{B}
225126,1. If the system of inequalities about $x$ $\l...,2\leqslant<3or-3\leqslant<-2
225127,"20.21. The numbers $1,2, \ldots, k^{2}$ are ar...",\frac{k(k^{2}+1)}{2}


In [39]:
# filter output if answer length < 10
def filter_answer_length(example):
    return len(example["answer"]) >= 200

filtered_df = df[df.apply(filter_answer_length, axis=1)]

In [40]:
filtered_df 

,question,answer
0,Find the sum of all three-digit positive integ...,To find the sum of all three-digit positive in...
1,"What is the largest number, all of whose digit...",We need to find the largest number consisting ...
2,"Two distinct primes, each greater than 20, are...",To find the least possible product of two dist...
3,A farmer has 100 meters of fencing and wants t...,Let's denote the length of the inner rectangul...
4,"If $\log (x^2y^2)= 3$ and $\log (xy^4) = 4$, ...",## Step 1: Use the properties of logarithms to...
...,...,...
11544565,Given the function $f(x) = e^x - \ln x (x > 0)...,h(x_1) = e^{\frac{1}{x_0}} - e^m \ln \frac{1}{...
11547637,"Let $S=\{1, 2, 3, \dots 2021\}$ and $f:S \to S...","\{43, 86, 129, 172, 215, 258, 301, 344, 387, ..."
11576267,3. (2 points) It is known that $\operatorname{...,"[\begin{pmatrix}\operatorname{tg}=1,\operatorn..."
11582531,3. For which positive integers $n$ is $\phi(n)...,"a) 1,2 \quad b) n \text{ such that } 8|n; 4|n ..."


In [20]:
cleaned_dataset.push_to_hub("vinhpx/math_natural_reasoning")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/359 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/359 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/331 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/vinhpx/math_natural_reasoning/commit/b90e4540eae549a74c8e5992e39c42948117db7b', commit_message='Upload dataset', commit_description='', oid='b90e4540eae549a74c8e5992e39c42948117db7b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vinhpx/math_natural_reasoning', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vinhpx/math_natural_reasoning'), pr_revision=None, pr_num=None)

In [21]:
cleaned_dataset

Dataset({
    features: ['question', 'answer', '__index_level_0__'],
    num_rows: 717281
})

In [22]:
df

,question,answer
0,Find the sum of all three-digit positive integ...,To find the sum of all three-digit positive in...
1,"What is the largest number, all of whose digit...",We need to find the largest number consisting ...
2,"Two distinct primes, each greater than 20, are...",To find the least possible product of two dist...
3,A farmer has 100 meters of fencing and wants t...,Let's denote the length of the inner rectangul...
4,"If $\log (x^2y^2)= 3$ and $\log (xy^4) = 4$, ...",## Step 1: Use the properties of logarithms to...
...,...,...
11627409,"Given the vectors $$\overrightarrow {a}=(1,1,x...",1
11627411,Given the following equations: 2+4=6; 8+10+12=...,\text{B}
11627412,1. If the system of inequalities about $x$ $\l...,2\leqslant<3or-3\leqslant<-2
11627413,"20.21. The numbers $1,2, \ldots, k^{2}$ are ar...",\frac{k(k^{2}+1)}{2}


In [23]:
concat_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 11627415
})